# Session 9: Modelling Planetary orbits

*Louise Dash louise.dash@ucl.ac.uk* (Last updated: 29/11/2018)

Our final task for this section will take computational modelling further: we will start with just one equation, Newton's law of universal gravitation, and from this develop a code that will model a planet orbiting a star.

### Why is this interesting?
In the previous example, the projectile, we had already derived the equations of motion analytically, so at any point in time $t$ we knew exactly the $x$- and $y$-coordinates of the ball.

In this example, we will specify our initial conditions (the planet's initial position and initial velocity), and then let the system evolve in time subject to the force between the planet and the star. We don't use any analytical equation to describe the equation of motion.

This also means we can explore how the initial conditions affect the motion of the planet, and find out more about the potential pitfalls of creating computational models!

### Newton's law of universal gravitation
Newton's law of universal gravitation states that the gravitational force between any two objects is proportional to their masses, and inversely proportional to the square of the distance between them:

$$F = \frac{G m_1 m_2}{r^2}$$

In SI units, the gravitational constant $G = 6.674 \times 10^{-11}$ N.m$^2$kg$^{-2}$ (this is very small - gravity is a very weak force!)

The image below illustrates Newton's law of universal gravitation. A point mass $m_1$ attracts another point mass $m_2$ by a
force $F_2$ pointing along the line intersecting both points. In the same way, $m_2$ attracts $m_1$ with an equal force $F_1$ pointing in the opposite direction. 
<img src='./NewtonsLawOfUniversalGravitation.svg'/>

(If you can't see an image above, either download it from Moodle, or take a look at the online version here: http://commons.wikimedia.org/wiki/File:NewtonsLawOfUniversalGravitation.svg )


Consider two masses $m_1$ and $m_2$, as shown in the figure. The gravitational force exerted on $m_1$ due to $m_2$ will be

$$
\mathbf{F}_{12} = - G  \frac{m_1 m_2}{|\mathbf{r}_{21}|^2} \hat{\mathbf{r}}_{21},
$$

where $\mathbf{r}_{21}$ is the position vector from $m_2$ to $m_1$, i.e. $\mathbf{r}_1 - \mathbf{r}_2$

Similarly, the force exerted on $m_2$ due to $m_1$ will be

$$\mathbf{F}_{21} = - G \frac{m_1 m_2}{|\mathbf{r}_{12}|^2} \hat{\mathbf{r}}_{12}$$

and clearly $\mathbf{F}_{12} = -\mathbf{F}_{21}$: the forces are equal and opposite, as they should be according to Newton's third law.

From Newton's second law, we also have $\mathbf{F} = m\mathbf{a}$, so therefore we can see that the accelerations experienced by $m_1$
due to $m_2$ and vice versa will be

$$\mathbf{a}_{1} = -\frac{G m_2}{r_{12}^2}\hat{\mathbf{r}}_{12}$$ 

$$\mathbf{a}_{2} = - \frac{G m_1}{r_{21}^2}\hat{\mathbf{r}}_{21}$$

These accelerations will not be equal in magnitude unless the objects have equal masses.

### Converting Newton's laws to a numerical model
Let's consider a system with two objects: a "star" with mass $M$ and a "planet" with mass $m$. Under normal circumstances clearly $M \gg m$ and so the acceleration experienced by the star due to the planet will be small compared to the acceleration experienced by the planet due to the star. 
The gravitational force on the planet due to the star is then

$$\mathbf{F}_{mM} = -G M m \frac{\hat{\mathbf{r}}_{Mm}}{|\mathbf{r}_{Mm}|^2}$$

To keep things simple, we will assume the acceleration on the star due to the planet is small enough that we can neglect it, and put the star in a fixed position at the origin of our co-ordinate system. Thus $\mathbf{r}_{Mm} = \mathbf{r}_m - \mathbf{r}_M$ becomes $\mathbf{r}_m$. We'll drop the now unnecessary subscript on $\mathbf{r}$, and replace the unit vector $\hat{\mathbf{r}}$ with $\mathbf{r}/|r|$, to give a simpler-looking expression:

$$\mathbf{F}_{mM} = -G M m \frac{\mathbf{r}}{|\mathbf{r}|^3}.$$

We can work out the motion of the planet straight from this, just by using the laws of motion:

$$  \mathbf{F}_{mM}  = m \mathbf{a} = m \frac{d \mathbf{v}}{d t} = m  \frac{d^2 \mathbf{r}}{d t^2}$$

In order to consider the motion of the planet around the star, we need to take the initial position and velocity of the planet as the
starting positions, so we need to integrate in order to calculate the force. There are several ways of numerical integration
you will learn about, but here we are just going to use the simplest, most naive method and convert
the differentials to small finite differences (as you will
learn later, this is definitely not the "best" way to do this!)

We therefore rewrite the force as
$$\mathbf{F}_{mM} = m \frac{d \mathbf{v}}{d t} \approx 
m\frac{\delta \mathbf{v}}{\delta t}$$

and then rearrange this to give the change in velocity of the planet in the time $\delta t$ as

$$\delta\mathbf{v} = \frac{\mathbf{F}_{Mm}}{m} \delta t = -G M
\frac{\mathbf{r}}{|\mathbf{r}|^3} \delta t$$

where $\delta \mathbf{v}$ and $\delta t$ are very small finite quantities. We can define $\delta t$ (usually known as the timestep) in our code as a parameter.

Similarly, we can write the change in the planet's position over the same $\delta t$ as
$$\delta \mathbf{r} = \mathbf{v} \delta t$$

Given our starting point of the initial position and velocity of the planet, and a fixed timestep $\delta t$, we can calculate how $\mathbf{r}$ and $\mathbf{v}$ change:

$$\mathbf{r}(t + \delta t)  = \mathbf{r}(t) + \delta \mathbf{r} $$

$$ \qquad  \qquad = \mathbf{r}(t) + \mathbf{v} \delta t $$

$$ \mathbf{v}(t + \delta t)  = \mathbf{v}(t) + \delta \mathbf{v} $$

$$  \qquad \qquad = \mathbf{v}(t) - \frac{GM \mathbf{r}}{|\mathbf{r|^3}} \delta t =  \mathbf{v}(t) - \frac{GM \hat{\mathbf{r}}}{|\mathbf{r|^2}} \delta t$$


Our outline algorithm will then look like this:

1. Input initial velocity, position and timestep
2. Calculate the change in velocity $\mathbf{v} \leftarrow \mathbf{v} - \frac{GM \mathbf{r}}{|\mathbf{r|^3}} \delta t$,  or equivalently $\mathbf{v} \leftarrow \mathbf{v} - \frac{GM \hat{\mathbf{r}}}{|\mathbf{r|^2}} \delta t$
3. Calculate the change in position: $\mathbf{r} \leftarrow \mathbf{r} + \mathbf{v} \delta t $
4. Repeat steps 2-3 ad libitum

## What to do:


1. Copy and paste the code template below into a new Jupyter notebook.
2. Add a suitable title and brief introduction.
3. Complete the code by implementing the algorithm above. In particular, note:

    * It is easier not to use realistic values for the physical quantities. This is why we've chosen units where G = 1, and we've chosen an arbitrary 1000:1 mass ratio for the star and the planet. When you have everything working right, you can try putting in "real" values for everything if you really want, but it's actually easier to explore the physics of the model with these simpler values.

    * I've suggested initial positions and velocities that will give a stable orbit if you choose a suitable timestep. You will need to experiment to see what works. 

    * When you've found a good timestep, vary the initial positions and velocities, and **choose an interesting set of initial conditions (different from the ones in the template!) for your uploaded code.**

    * It is possible to do all this in 10-20 lines of code (including comments etc). If your code ends up much longer than this, you're probably overcomplicating things.

### CHALLENGE!

*(As last week, optional, but with a small amount of credit. Only minimal help from demonstrators available - you're on your own now!)* 

Leave your "basic" code as it is, but copy and paste it into a new code cell and try adding one or more of these (some are harder than others!)
* an "arrow" object to represent the vector force on the planet due to the star
* add a second planet to the system - both planets will need to interact with each other. 
* a moon orbiting the planet (hard).

Make sure your code fully describes what you're doing (or trying to do!) with **both comments _and_ text cells**

### New commands and handy hints:

Note the new vpython command that appear in the code template:
* <tt> mag </tt> : finds the magnitude of a vector

You can find more information on this, and other useful vector functions, here: http://www.glowscript.org/docs/VPythonDocs/vector.html

You may also want to put some temporary `print()` functions within your loop while you're debugging. Sometimes vpython doesn't output these when you'd expect. If you're using vpython, and your `print()` statements aren't giving you the output you'd expect, try adding "flush=True", like this:

                                print("my value is", value, flush=True)
This will force python to "flush" all the output to the screen as soon as it gets to the `print()`. Have a look at the docstring for `print()` for more details.   

In [ ]:
# Template code - copy and paste this into a new Jupyter Notebook

from vpython import sphere, vector, color, rate, mag, canvas, label

canvas(width = 640, height = 480) # slightly bigger than default, adjust if you have small screen.

dt = 0.01      # timestep
step = 1       # loop counter
maxstep = 2000 # maximum number of steps

#  Define the star, planets and constants
M = 1000       # mass of star (in units where G = 1)
mass = 1       # mass of planet (")
posn = vector(0,1,0)               # initial position vector of Planet
Planet = sphere(pos=posn,radius=0.05*mass,color=color.blue,make_trail=True)
Planet.trail_color = color.white # change the trail colour to white, or any colour you fancy
Star = sphere(pos=vector(0,0,0),radius=0.1,color=color.yellow)
vel = vector(-25, 0, 0)               # initial velocity of Planet

while step <= maxstep:

   ### YOUR CODE HERE ###

print("end of program")